# Personal HTCondor Pool Demo

A Personal HTCondor Pool is an HTCondor Pool that has a single Owner, who is defined by being:
- The pool’s only Administrator.
- The only Submitter who is allowed to submit to the pool’s SchedD.
- The Resource Owner of all resources managed by the pool.

In [ ]:
import logging
from pathlib import Path

import htcondor
import classad

from personal import PersonalPool

In [ ]:
logging.basicConfig(format="%(asctime)s ~ %(msg)s", level=logging.DEBUG)

We can start a personal pool by making a `PersonalPool` and calling its `start()` method.

In [ ]:
p = PersonalPool().start().detach()
p.stop()

In [ ]:
pool = PersonalPool()
pool

In [ ]:
pool.start()
pool

In [ ]:
print(pool.get_config_val("LOCAL_DIR"))

In [ ]:
print(pool.run_command(["condor_config_val", "-dump", "LOCAL_DIR"]).stdout)

In [ ]:
with pool.collector() as coll:
    print(coll.query()[0])

In [ ]:
sub = htcondor.Submit(
    executable = "/bin/sleep",
    arguments = "${ProcID}s",
)

with pool.schedd() as schedd:
    with schedd.transaction() as txn:
        cluster_id = sub.queue(txn, 5)

    print(f"ClusterID is {cluster_id}")
    
    for ad in schedd.query(
        constraint = f"ClusterID == {cluster_id}", 
        attr_list = ["ClusterID", "ProcID", "JobStatus"]
    ):
        print(repr(ad))

In [ ]:
pool.stop()

Using the pool as a context manager makes it start/stop automatically.

In [ ]:
with PersonalPool() as pool:
    print(pool.get_config_val("LOCAL_DIR"))
    condor_status = pool.run_command(["condor_status"]).stdout
    print(condor_status)

You can't have two personal pools with the same local directory at the same time.

In [ ]:
Stoppingp1 = PersonalPool(local_dir = Path.cwd() / "target")
p2 = PersonalPool(local_dir = Path.cwd() / "target")

In [ ]:
print(p1)
print(p2)

In [ ]:
p1.start()

In [ ]:
p2.start()

In [ ]:
p1.stop()

In [ ]:
p2.start()
p2.stop()